In [ ]:
(* Binary trees *)
type 'a tree = Lf | Br of 'a * 'a tree * 'a tree

In [ ]:
let rec sumtree = function
	| Lf -> 0
	| Br (n , t1, t2) -> n + sumtree t1 + sumtree t2

In [ ]:
let rec ftree k n =
	if n = 0 then Lf
	else Br (k, ftree (2*k) (n - 1), ftree (2*k + 1) (n - 1))

In [ ]:
(* Detele function *)

(* Pops out the largest key-value pair from a tree *)let rec pop_max = function
	| Lf -> (None, Lf)
	| Br (n, t1, Lf) -> (Some n, t1)
	| Br (n, t1, t2) -> 
		let (max, t2') = pop_max t2 in
			(max, Br(n, t1, t2'))

let rec delete tr k = match tr with
	| Lf -> Lf
	| Br ((a , x) , t1 , t2) -> 
		if k < a then Br ((a , x) , delete t1 k , t2)
		else if k > a then Br ((a , x) , t1 , delete t2 k)
		else match pop_max t1 with
			| None, _ -> t2   (* t1 must be Lf *)
			| Some max, t1' -> Br(max, t1', t2)

In [ ]:
(* Verify binary search tree invariant *)
let rec tree_lt tr n = match tr with
	| Lf -> true
	| Br ((k, v), t1, t2) -> k < n && tree_lt t1 n && tree_lt t2 n

let rec tree_gt tr n = match tr with
	| Lf -> true
	| Br ((k, v), t1, t2) -> k > n && tree_gt t1 n && tree_gt t2 n

let rec valid_search_tree = function
	| Lf -> true
	| Br ((k, v), t1, t2) -> 
		tree_lt t1 k && tree_gt t2 k 
		&& valid_search_tree t1 && valid_search_tree t2		

In [ ]:
(* An optimized way for this: keeping ranges *)
let in_bound k lb ub = match (lb, ub) with
		| None, None -> true
		| None, Some n -> k < n
		| Some m, None -> k > m
		| Some m, Some n -> k > m && k < n

let valid_search_tree_fast tr =
	let rec valid lb ub = function 
		| Lf -> true
		| Br ((k, v), t1, t2) -> 
			in_bound k lb ub &&
			valid lb (Some k) t1 &&
			valid (Some k) ub t2
	in
		valid None None tr

In [ ]:
(* Detele first item from array: shift, reshape, then decrement key. *)

(* Shift: move the left-most branch up a place *)
let rec shift = function
	| Lf -> Lf
	| Br (_ , Lf , t2) -> t2
	| Br (_ , Br((a , x) , t1 , t2) , Lf) -> Br((a , x) , t1 , t2)
	| Br (_ , Br((a , x) , t1 , t2) , t3) ->
		Br ((a , x) , shift (Br((a , x) , t1 , t2)) , t3)

(* Reshape: swap left and right branches *)
let rec reshape = function
	| Lf -> Lf
	| Br ((k, v), t1, t2) -> Br((k, v), t2, reshape t1)

(* Dec_key: decrement key by 1 *)
let rec dec_key = function
	| Lf -> Lf
	| Br ((k, v), t1, t2) -> Br((k - 1, v), dec_key t1, dec_key t2)

let remove_first arr = dec_key (reshape (shift arr)) 

(* Verification, optimized version: left as exercise. *)